In [79]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [80]:
import pandas as pd

In [81]:
train = pd.read_csv('../data/weather_classification_data.csv')
train.head(1)

,Temperature,Humidity,Wind Speed,Precipitation (%),Cloud Cover,Atmospheric Pressure,UV Index,Season,Visibility (km),Location,Weather Type
0,14.0,73,9.5,82.0,partly cloudy,1010.82,2,Winter,3.5,inland,Rainy


In [82]:
from auto_ml_kinder import pre_processing as pp

In [83]:
train.columns = train.columns.str.lower().str.replace(' ', '_')

In [84]:
train.head(1)

,temperature,humidity,wind_speed,precipitation_(%),cloud_cover,atmospheric_pressure,uv_index,season,visibility_(km),location,weather_type
0,14.0,73,9.5,82.0,partly cloudy,1010.82,2,Winter,3.5,inland,Rainy


In [85]:
label_encode:list[pp.PreLabelEncoderConfig] = []
label_encode.append(pp.PreLabelEncoderConfig(
    column_name='cloud_cover',
    label_encoding=list({pp.LabelEncodingDict(key, idx + 1) for idx, key in enumerate(train.cloud_cover.value_counts().keys())}))
    )
label_encode.append(pp.PreLabelEncoderConfig(
    column_name='season',
    label_encoding=list({pp.LabelEncodingDict(key, idx + 1) for idx, key in enumerate(train.season.value_counts().keys())}))
    )
label_encode.append(pp.PreLabelEncoderConfig(
    column_name='location',
    label_encoding=list({pp.LabelEncodingDict(key, idx + 1) for idx, key in enumerate(train.location.value_counts().keys())}))
    )
label_encode.append(pp.PreLabelEncoderConfig(
    column_name='weather_type',
    label_encoding=list({pp.LabelEncodingDict(key, idx + 0) for idx, key in enumerate(train.weather_type.value_counts().keys())}))
    )
# label_encode.append(pp.PreLabelEncoderConfig(
#     column_name='restaurant_type',
#     label_encoding=list({pp.LabelEncodingDict(key, idx + 1) for idx, key in enumerate(train.restaurant_type.value_counts().keys())}))
#     )
# label_encode.append(pp.PreLabelEncoderConfig(
#     column_name='cuisine',
#     label_encoding=list({pp.LabelEncodingDict(key, idx + 1) for idx, key in enumerate(train.cuisine.value_counts().keys())}))
#     )

numeric_col_changer:list[pp.PreNumericColDataChangeConfig] = []
# numeric_col_changer.append(pp.PreNumericColDataChangeConfig(col_name='facebook_popularity_quotient',data_type=int))
# numeric_col_changer.append(pp.PreNumericColDataChangeConfig(col_name='instagram_popularity_quotient',data_type=int))
# numeric_col_changer.append(pp.PreNumericColDataChangeConfig(col_name='overall_restaurant_rating',data_type=int))
# numeric_col_changer.append(pp.PreNumericColDataChangeConfig(col_name='live_music_rating',data_type=int))
# numeric_col_changer.append(pp.PreNumericColDataChangeConfig(col_name='ambience',data_type=int))
# numeric_col_changer.append(pp.PreNumericColDataChangeConfig(col_name='resturant_tier',data_type=int))

model_config = pp.PreProcessingConfig(
    encoding_dummies=[],
    exclude_columns=[],
    label_encode=label_encode,
    numeric_cols_data_changer=numeric_col_changer,
    target_column='weather_type',
    is_classification=True
)

In [86]:
pre_processed_df = pp.process(train,model_config)

Dropping columns whose values are nullable greater than 0.7
Removing unusable values like " " and "?"
Filling NA values with median and mode for numeria and non-numeric columns.
Changing datatype of numeric columns.
Performing label encoding on columns provided.
Encoding using dummies.


In [87]:
pre_processed_df.head()

,temperature,humidity,wind_speed,precipitation_(%),cloud_cover,atmospheric_pressure,uv_index,season,visibility_(km),location,weather_type
0,14.0,73,9.5,82.0,2,1010.82,2,1,3.5,1,0
1,39.0,96,8.5,71.0,2,1011.43,7,2,10.0,1,1
2,30.0,64,7.0,16.0,3,1018.72,5,2,5.5,2,2
3,38.0,83,1.5,82.0,3,1026.25,7,2,1.0,3,2
4,27.0,74,17.0,66.0,1,990.67,1,1,2.5,2,0


In [88]:
from auto_ml_kinder import model_training_data_prep as dp
from auto_ml_kinder import model_training_helper as mth
from auto_ml_kinder import model_list_helper as mlh

In [89]:
data = dp.ModelTrainingData(pre_processed_df, dp.ScalerType.STANDARD_SCALER, .95,use_pca=False,use_polynomials=False,use_feature_selection=False,create_clustering_feature_and_no_of_clusters=(False,3),is_classification=True)

Total columns being used after all data transformations: 10
Since num of classes is 4 transforming Y_(test/train/val)_neural_network variables to categorical.


In [90]:
trainer = mth.ModelTrainer(data=data)

In [91]:
trainer.perform_operation_classification(exclude_models=[
    mlh.ModelAndParamClassifiction.SVC_Classification,
    mlh.ModelAndParamClassifiction.DecisionTree_Classifiction,
    mlh.ModelAndParamClassifiction.RandomForest_Classifiction,
    mlh.ModelAndParamClassifiction.GradientBoosting_Classifiction,
    mlh.ModelAndParamClassifiction.KNeighbors_Classifiction
    ],permutate_n_less_column=2)
trainer.performance_df

Total Permutations: 56
Total Permutations: 56


,model_name,score,RMSE,total_columns,scaler_type,selected_features,num_of_clusters_if_used,accuracy,f1_score
0,Logistic_Regression,0.860606,None,8,StandardScaler,"temperature, humidity, wind_speed, precipitati...",0,0.860606,0.859757
1,Logistic_Regression,0.880303,None,8,StandardScaler,"temperature, humidity, wind_speed, precipitati...",0,0.880303,0.880040
2,Logistic_Regression,0.863258,None,8,StandardScaler,"temperature, humidity, wind_speed, precipitati...",0,0.863258,0.862592
3,Logistic_Regression,0.850758,None,8,StandardScaler,"temperature, humidity, wind_speed, precipitati...",0,0.850758,0.849931
4,Logistic_Regression,0.833712,None,8,StandardScaler,"temperature, humidity, wind_speed, precipitati...",0,0.833712,0.832474
...,...,...,...,...,...,...,...,...,...
107,Ridge_Classifiction,0.778788,None,9,StandardScaler,"temperature, humidity, wind_speed, cloud_cover...",0,0.778788,0.771290
108,Ridge_Classifiction,0.795455,None,9,StandardScaler,"temperature, humidity, precipitation_(%), clou...",0,0.795455,0.792466
109,Ridge_Classifiction,0.798864,None,9,StandardScaler,"temperature, wind_speed, precipitation_(%), cl...",0,0.798864,0.795266
110,Ridge_Classifiction,0.751894,None,9,StandardScaler,"humidity, wind_speed, precipitation_(%), cloud...",0,0.751894,0.748102


In [92]:
data = dp.ModelTrainingData(pre_processed_df, 
                            dp.ScalerType.QUANTILE_TRANSFORMER,
                              .95,
                              use_pca=False,
                              use_polynomials=False,
                              use_feature_selection=False,
                              create_clustering_feature_and_no_of_clusters=(False,3),
                              is_classification=True)

Total columns being used after all data transformations: 10
Since num of classes is 4 transforming Y_(test/train/val)_neural_network variables to categorical.


In [93]:
trainer.data = data

In [94]:
trainer.perform_operation_classification(exclude_models=[
    mlh.ModelAndParamClassifiction.SVC_Classification,
    mlh.ModelAndParamClassifiction.DecisionTree_Classifiction,
    mlh.ModelAndParamClassifiction.RandomForest_Classifiction,
    mlh.ModelAndParamClassifiction.GradientBoosting_Classifiction,
    mlh.ModelAndParamClassifiction.KNeighbors_Classifiction,
    mlh.ModelAndParamClassifiction.Ridge_Classifiction
    ],permutate_n_less_column=0)
trainer.performance_df

Total Permutations: 1


,model_name,score,RMSE,total_columns,scaler_type,selected_features,num_of_clusters_if_used,accuracy,f1_score
0,Logistic_Regression,0.860606,None,8,StandardScaler,"temperature, humidity, wind_speed, precipitati...",0,0.860606,0.859757
1,Logistic_Regression,0.880303,None,8,StandardScaler,"temperature, humidity, wind_speed, precipitati...",0,0.880303,0.880040
2,Logistic_Regression,0.863258,None,8,StandardScaler,"temperature, humidity, wind_speed, precipitati...",0,0.863258,0.862592
3,Logistic_Regression,0.850758,None,8,StandardScaler,"temperature, humidity, wind_speed, precipitati...",0,0.850758,0.849931
4,Logistic_Regression,0.833712,None,8,StandardScaler,"temperature, humidity, wind_speed, precipitati...",0,0.833712,0.832474
...,...,...,...,...,...,...,...,...,...
108,Ridge_Classifiction,0.795455,None,9,StandardScaler,"temperature, humidity, precipitation_(%), clou...",0,0.795455,0.792466
109,Ridge_Classifiction,0.798864,None,9,StandardScaler,"temperature, wind_speed, precipitation_(%), cl...",0,0.798864,0.795266
110,Ridge_Classifiction,0.751894,None,9,StandardScaler,"humidity, wind_speed, precipitation_(%), cloud...",0,0.751894,0.748102
111,Ridge_Classifiction,0.803788,None,10,StandardScaler,"temperature, humidity, wind_speed, precipitati...",0,0.803788,0.801007


In [99]:
trainer.performance_df.loc[trainer.performance_df['score'].idxmax()]

model_name                                                     NeuralNetwork
score                                                               0.890152
RMSE                                                                    None
total_columns                                                             10
scaler_type                                              QuantileTransformer
selected_features          temperature, humidity, wind_speed, precipitati...
num_of_clusters_if_used                                                    0
accuracy                                                            0.890152
f1_score                                                            0.890383
Name: 113, dtype: object

In [96]:
trainer.perform_neural_network_classification(totalExperiments=10,
                                          params=mth.NeuralNetwork_BayesianOptimization_Params(
                                              neurons_min_max=(32,128)
                                              ,batch_size_min_max=(32,32)
                                              ,dropout_rate_min_max=(.3,.7)
                                              ,epochs_min_max=(50,100)
                                              ,hidden_layers_min_max=(2,4)
                                              ,learning_rate_min_max=(.001,.1)
                                              ,normalization_min_max=(0,1)
                                              ,dropout_min_max=(0,1)
                                              ,activation_min_max=(0,3)
                                              ))

|   iter    |  target   | activa... | batch_... |  dropout  | dropou... |  epochs   | hidden... | learni... |  neurons  | normal... | optimizer |
-------------------------------------------------------------------------------------------------------------------------------------------------
Neurons: 72||Total Columns:10||Normalization:False||Activation:selu||Total hidden layers:2||Dropout used:False||Dropout rate if used:0.6077049890092492||Optimizer:adadelta||Epochs:65||Batch_size:32

| 1         | 0.6937    | 1.837     | 32.0      | 0.4361    | 0.6077    | 64.77     | 2.298     | 0.003225  | 72.34     | 0.2387    | 2.026     |
Neurons: 43||Total Columns:10||Normalization:False||Activation:elu||Total hidden layers:3||Dropout used:False||Dropout rate if used:0.567840095298652||Optimizer:nadam||Epochs:81||Batch_size:32

| 2         | 0.8915    | 2.972     | 32.0      | 0.08119   | 0.5678    | 81.06     | 2.549     | 0.04716   | 43.36     | 0.07396   | 5.405     |
Neurons: 35||Total Colu

In [97]:
trainer.neural_network_best_model_classification(epochs=500)

Neurons: 90
Total Columns:10
Normalization:False
Activation:<keras.src.layers.activation.leaky_relu.LeakyReLU object at 0x000001FC7A431030>
Total hidden layers:2
Dropout used:True
Dropout rate if used:0.6452415184846682
Optimizer:rmsprop
Epochs:500
Batch_size:32

Epoch 1/500
231/231 [==============================] - 1s 3ms/step - loss: 11.1657 - accuracy: 0.3095 - val_loss: 1.1771 - val_accuracy: 0.6001
Epoch 2/500
231/231 [==============================] - 1s 2ms/step - loss: 1.6418 - accuracy: 0.4108 - val_loss: 0.9726 - val_accuracy: 0.7345
Epoch 3/500
231/231 [==============================] - 1s 2ms/step - loss: 1.2767 - accuracy: 0.5073 - val_loss: 0.8863 - val_accuracy: 0.7225
Epoch 4/500
231/231 [==============================] - 1s 2ms/step - loss: 1.1011 - accuracy: 0.5805 - val_loss: 0.7982 - val_accuracy: 0.7914
Epoch 5/500
231/231 [==============================] - 1s 2ms/step - loss: 0.9729 - accuracy: 0.6368 - val_loss: 0.8543 - val_accuracy: 0.7175
Epoch 6/500
231/231 

In [98]:
trainer.performance_df

,model_name,score,RMSE,total_columns,scaler_type,selected_features,num_of_clusters_if_used,accuracy,f1_score
0,Logistic_Regression,0.860606,None,8,StandardScaler,"temperature, humidity, wind_speed, precipitati...",0,0.860606,0.859757
1,Logistic_Regression,0.880303,None,8,StandardScaler,"temperature, humidity, wind_speed, precipitati...",0,0.880303,0.880040
2,Logistic_Regression,0.863258,None,8,StandardScaler,"temperature, humidity, wind_speed, precipitati...",0,0.863258,0.862592
3,Logistic_Regression,0.850758,None,8,StandardScaler,"temperature, humidity, wind_speed, precipitati...",0,0.850758,0.849931
4,Logistic_Regression,0.833712,None,8,StandardScaler,"temperature, humidity, wind_speed, precipitati...",0,0.833712,0.832474
...,...,...,...,...,...,...,...,...,...
109,Ridge_Classifiction,0.798864,None,9,StandardScaler,"temperature, wind_speed, precipitation_(%), cl...",0,0.798864,0.795266
110,Ridge_Classifiction,0.751894,None,9,StandardScaler,"humidity, wind_speed, precipitation_(%), cloud...",0,0.751894,0.748102
111,Ridge_Classifiction,0.803788,None,10,StandardScaler,"temperature, humidity, wind_speed, precipitati...",0,0.803788,0.801007
112,Logistic_Regression,0.884091,None,10,QuantileTransformer,"temperature, humidity, wind_speed, precipitati...",0,0.884091,0.883795
